In [ ]:
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from Parser import parse_frames
from scipy.signal import medfilt
from Functions import analyse_each_rep, find_extremas
from JointAngles import JointAngles
import numpy as np
import glob
import os
import pickle
import pandas as pd
from functools import reduce


In [ ]:
# input_folder = "C:\\Users\\altaok\\Documents\\GitHub\\IndividualProject\\keypoints_for_all"
input_folder = 'C:\\Users\\ak5u16\\Desktop\\IndividualProject\\keypoints_for_all\\shoulder press'
folder_paths = glob.glob(os.path.join(input_folder, 'shoulder_press*'))


def fill_dataframe(keypoints_paths):
    angle_label_dict = {}
    data = {}
    labels = []
    df_list = []
    for folder in folder_paths:
        video_name = os.path.basename(folder)
        label = 0 if 'correct' in folder else 1
        frame_poses = parse_frames(folder)
        joint_angles = JointAngles('shoulder press', frame_poses)

        upArm_trunk_angles = np.array(joint_angles.upArm_trunk_angles, dtype=np.float)
        upArm_trunk_angles = np.nan_to_num(upArm_trunk_angles)
        upArm_trunk_angles_filtered = medfilt(medfilt(upArm_trunk_angles, 5), 5)

        
        upArm_forearm_angles = np.array(joint_angles.upArm_forearm_angles, dtype=np.float)
        upArm_forearm_angles = np.nan_to_num(upArm_forearm_angles)
        upArm_forearm_angles_filtered = medfilt(medfilt(upArm_forearm_angles, 5), 5)
       

        trunk_knee_angles = np.array(joint_angles.trunk_knee_angles, dtype=np.float)
        trunk_knee_angles = np.nan_to_num(trunk_knee_angles)
        trunk_knee_angles_filtered = medfilt(medfilt(trunk_knee_angles, 5), 5)
        
        #number of reps 
        upArm_forearm_maximas, indexesmax3 = find_extremas(upArm_forearm_angles_filtered) 
       
        if upArm_forearm_maximas.size == 0:
            s1 = pd.Series([upArm_forearm_angles_filtered, upArm_trunk_angles_filtered,  trunk_knee_angles_filtered], name='Angle_array')
            s2 = pd.Series([label, label, label], name='Label')
            df = pd.concat([s1,s2], axis=1)           
            df_list.append(df)
           
            
        else:         
            # Extract rep angles
            each_rep_angles = analyse_each_rep('dataset', trunk_knee_angles_filtered, upArm_forearm_angles_filtered, upArm_trunk_angles_filtered, upArm_forearm_maximas)            
            for n in each_rep_angles:
                labels.append(label)    
                               
            s1 = pd.Series(each_rep_angles, name='Angle_array')
            s2 = pd.Series(labels, name='Label')
            df = pd.concat([s1,s2], axis=1)
        
            df_list.append(df)
            # Erase labels
            labels = []
            
    
    if len(df_list) > 1:
        df_final = pd.concat(df_list, axis=0).reset_index(drop=True)
       
    return df_final


        